In [29]:
import os
import shutil
import subprocess

In [30]:
strTmpDir = './dataset_tmp'
os.makedirs(strTmpDir, exist_ok=True)
subprocess.check_call(["wget", "-O", os.path.join(strTmpDir, "cifar-10-python.tar.gz"), "http://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz",])
subprocess.check_call(["tar", "xfvz", os.path.join(strTmpDir, "cifar-10-python.tar.gz"), "-C", strTmpDir, ])
os.makedirs(os.path.join(strTmpDir, "data"), exist_ok=True)
subprocess.check_call(["mv", os.path.join(strTmpDir, "cifar-10-batches-py"), os.path.join(strTmpDir, "data"),])
subprocess.check_call(["rm", os.path.join(strTmpDir, "cifar-10-python.tar.gz"),])

--2023-03-02 13:03:48--  http://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz
Resolving www.cs.toronto.edu (www.cs.toronto.edu)... 128.100.3.30
Connecting to www.cs.toronto.edu (www.cs.toronto.edu)|128.100.3.30|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 170498071 (163M) [application/x-gzip]
Saving to: ‘./dataset_tmp/cifar-10-python.tar.gz’

     0K .......... .......... .......... .......... ..........  0%  125K 22m8s
    50K .......... .......... .......... .......... ..........  0%  251K 16m36s
   100K .......... .......... .......... .......... ..........  0% 1.06M 11m55s
   150K .......... .......... .......... .......... ..........  0%  316K 11m7s
   200K .......... .......... .......... .......... ..........  0%  249K 11m7s
   250K .......... .......... .......... .......... ..........  0%  251K 11m6s
   300K .......... .......... .......... .......... ..........  0%  250K 11m6s
   350K .......... .......... .......... .......... ..........  0%  25

cifar-10-batches-py/
cifar-10-batches-py/data_batch_4
cifar-10-batches-py/readme.html
cifar-10-batches-py/test_batch
cifar-10-batches-py/data_batch_3
cifar-10-batches-py/batches.meta
cifar-10-batches-py/data_batch_2
cifar-10-batches-py/data_batch_5
cifar-10-batches-py/data_batch_1


0

In [31]:
if os.path.isdir(strTmpDir): shutil.rmtree(strTmpDir)

In [47]:
from sagemaker.pytorch.processing import PyTorchProcessor

In [48]:
PyTorchProcessor?

Init signature:
PyTorchProcessor(
    framework_version: str,
    role: str,
    instance_count: Union[int, sagemaker.workflow.entities.PipelineVariable],
    instance_type: Union[str, sagemaker.workflow.entities.PipelineVariable],
    py_version: str = 'py3',
    image_uri: Union[str, sagemaker.workflow.entities.PipelineVariable, NoneType] = None,
    command: Optional[List[str]] = None,
    volume_size_in_gb: Union[int, sagemaker.workflow.entities.PipelineVariable] = 30,
    volume_kms_key: Union[str, sagemaker.workflow.entities.PipelineVariable, NoneType] = None,
    output_kms_key: Union[str, sagemaker.workflow.entities.PipelineVariable, NoneType] = None,
    code_location: Optional[str] = None,
    max_runtime_in_seconds: Union[int, sagemaker.workflow.entities.PipelineVariable, NoneType] = None,
    base_job_name: Optional[str] = None,
    sagemaker_session: Optional[sagemaker.session.Session] = None,
    env: Optional[Dict[str, Union[str, sagemaker.workflow.entities.PipelineVaria

In [32]:
nModels=30

In [33]:
for nIdx in range(nModels):
    strPipelineName = f"train-pipeline-model-{nIdx+1}"
    !aws sagemaker delete-pipeline --pipeline-name $strPipelineName 

{
    "PipelineArn": "arn:aws:sagemaker:ap-northeast-2:419974056037:pipeline/train-pipeline-model-1"
}
{
    "PipelineArn": "arn:aws:sagemaker:ap-northeast-2:419974056037:pipeline/train-pipeline-model-2"
}
{
    "PipelineArn": "arn:aws:sagemaker:ap-northeast-2:419974056037:pipeline/train-pipeline-model-3"
}
{
    "PipelineArn": "arn:aws:sagemaker:ap-northeast-2:419974056037:pipeline/train-pipeline-model-4"
}
{
    "PipelineArn": "arn:aws:sagemaker:ap-northeast-2:419974056037:pipeline/train-pipeline-model-5"
}
{
    "PipelineArn": "arn:aws:sagemaker:ap-northeast-2:419974056037:pipeline/train-pipeline-model-6"
}
{
    "PipelineArn": "arn:aws:sagemaker:ap-northeast-2:419974056037:pipeline/train-pipeline-model-7"
}
{
    "PipelineArn": "arn:aws:sagemaker:ap-northeast-2:419974056037:pipeline/train-pipeline-model-8"
}
{
    "PipelineArn": "arn:aws:sagemaker:ap-northeast-2:419974056037:pipeline/train-pipeline-model-9"
}
{
    "PipelineArn": "arn:aws:sagemaker:ap-northeast-2:419974056037:pipel

In [56]:
import time
from sagemaker import get_execution_role
from sagemaker.sklearn.processing import SKLearnProcessor
from sagemaker.processing import ProcessingInput, ProcessingOutput
role = get_execution_role()

In [73]:
sklearn_processor = SKLearnProcessor(
    framework_version="1.0-1",
    role=role,
    instance_type="local",
    #instance_type=self.args.config.get_value("DEPLOY", "processing_instance_type"),
    instance_count=1,
    base_job_name="deploy", # bucket에 보이는 이름 (pipeline으로 묶으면 pipeline에서 정의한 이름으로 bucket에 보임)
    #sagemaker_session=self.pipeline_session ## Add
)

INFO:botocore.credentials:Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole
INFO:sagemaker.image_uris:Defaulting to only available Python version: py3


In [74]:
prefix_deploy = "/opt/ml/processing/"
endpoint_name = f"endpoint-TEST-{int(time.time())}"
sklearn_processor.run(
    code='../sources/deploy/deploy.py',
    inputs=[
        ProcessingInput(
            input_name="inference-code",
            source="../sources/deploy/inference.py",
            destination=os.path.join(prefix_deploy, "input", "inference-code")
        ),
        ProcessingInput(
            input_name="requirements",
            source='../sources/deploy/requirements.txt',
            destination=os.path.join(prefix_deploy, "input", "requirements")
        ),
    ],
    arguments=["--prefix_deploy", prefix_deploy, "--region", "ap-northeast-2", \
               "--model_server_workers", str(1), \
               "--instance_type", "ml.g4dn.xlarge", \
               "--initial_instance_count", str(1), \
               "--image_uri", "763104351884.dkr.ecr.ap-northeast-2.amazonaws.com/pytorch-training:1.12.1-gpu-py38", \
               "--model_data", "s3://sagemaker-ap-northeast-2-419974056037/SM-BYOM-PIPELINE-TRAIN-SAMPLE-model-230304/pipelines-x67ty5zj8zml-TrainingProcess-sFBJD62IXF/output/model.tar.gz", \
               "--model_name", "modelname", \
               "--endpoint_name", "endpointname1", \
               "--execution_role", role, \
               "--framework_version", "1.12.1", \
               "--py_version", "py38"],
    
    job_name="deploy",

)

INFO:botocore.credentials:Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole
INFO:sagemaker:Creating processing-job with name deploy
INFO:sagemaker.local.local_session:Starting processing job
INFO:botocore.credentials:Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole
INFO:sagemaker.local.image:No AWS credentials found in session but credentials from EC2 Metadata Service are available.
INFO:sagemaker.local.image:docker compose file: 
networks:
  sagemaker-local:
    name: sagemaker-local
services:
  algo-1-n2lkn:
    container_name: v7ctz9uvjb-algo-1-n2lkn
    entrypoint:
    - python3
    - /opt/ml/processing/input/code/deploy.py
    - --prefix_deploy
    - /opt/ml/processing/
    - --region
    - ap-northeast-2
    - --model_server_workers
    - '1'
    - --instance_type
    - ml.g4dn.xlarge
    - --initial_instance_count
    - '1'
    - --image_uri
    - 763104351884.dkr.ecr.ap-northeast-2.amazonaws.com/pytorch-training:1.12.1-gpu-py38
    - --mod

Creating v7ctz9uvjb-algo-1-n2lkn ... 
Creating v7ctz9uvjb-algo-1-n2lkn ... done
Attaching to v7ctz9uvjb-algo-1-n2lkn
v7ctz9uvjb-algo-1-n2lkn | Received arguments Namespace(endpoint_name='endpointname1', execution_role='arn:aws:iam::419974056037:role/service-role/AmazonSageMaker-ExecutionRole-20221206T163436', framework_version='1.12.1', image_uri='763104351884.dkr.ecr.ap-northeast-2.amazonaws.com/pytorch-training:1.12.1-gpu-py38', initial_instance_count='1', instance_type='ml.g4dn.xlarge', model_data='s3://sagemaker-ap-northeast-2-419974056037/SM-BYOM-PIPELINE-TRAIN-SAMPLE-model-230304/pipelines-x67ty5zj8zml-TrainingProcess-sFBJD62IXF/output/model.tar.gz', model_name='modelname', model_server_workers='1', prefix_deploy='/opt/ml/processing/', py_version='py38', region='ap-northeast-2')
v7ctz9uvjb-algo-1-n2lkn | Requirement already satisfied: pip in /miniconda3/lib/python3.8/site-packages (22.3.1)
v7ctz9uvjb-algo-1-n2lkn | Collecting pip
v7ctz9uvjb-algo-1-n2lkn |   Downloading pip-23.0.1

RuntimeError: Failed to run: ['docker-compose', '-f', '/tmp/tmpk8megn8o/docker-compose.yaml', 'up', '--build', '--abort-on-container-exit']